In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
cities_data = pd.read_csv("resources\city_weather.csv")
cities_data = cities_data.drop('Unnamed: 0', axis=1)
cities_data.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,san patricio,28.0170,-97.5169,84.04,85.0,2.0,3.56,US,1.627466e+09
1,port alfred,-33.5906,26.8910,64.56,65.0,85.0,6.22,ZA,1.627466e+09
2,isangel,-19.5500,169.2667,69.82,66.0,3.0,11.21,VU,1.627466e+09
3,taoudenni,22.6783,-3.9836,105.40,14.0,100.0,11.63,ML,1.627466e+09
4,rikitea,-23.1203,-134.9692,74.16,82.0,100.0,14.83,PF,1.627466e+09


In [4]:
# Store lat and lon locations
locations = cities_data[['Lat', 'Lon']]

# Store humidity values
humidity = cities_data['Humidity']

# Configure maps
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

# Plot heatmaps
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=humidity.max())

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Narrow the selection of data

data_selection = (
                (cities_data['Max Temp'] > 70) &
                (cities_data['Max Temp'] < 80) &
                (cities_data['Wind Speed'] < 10) &
                (cities_data['Cloudiness'] == 0) &
                (cities_data['Humidity'] < 60)
)

ideal_cities = cities_data.loc[data_selection]
ideal_cities

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,lata,40.1629,-8.3327,74.35,40.0,0.0,5.97,PT,1.627466e+09
168,dingli,35.8614,14.3822,79.66,57.0,0.0,6.49,MT,1.627466e+09
192,cortes,41.9395,-1.4396,76.03,49.0,0.0,8.93,ES,1.627466e+09
230,morondava,-20.2833,44.2833,78.44,37.0,0.0,8.23,MG,1.627466e+09
297,bueu,42.3246,-8.7850,74.35,56.0,0.0,5.75,ES,1.627466e+09
361,komatipoort,-25.4332,31.9548,75.06,26.0,0.0,8.25,ZA,1.627466e+09
550,ankazobe,-18.3167,47.1167,70.21,21.0,0.0,7.70,MG,1.627466e+09


In [13]:
# Add column to dataframe
ideal_cities['Hotel name'] = ""

# Use Google API to get hotel data
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters for search
params = {
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}

for index, row in ideal_cities.iterrows():
    params["location"] = f"{row['Lat']},{row['Lon']}"
    
    response = requests.get(base_url, params=params).json()
    # Returns first hotel in Google's list. May not necessarily be the closest!
    try:
        hotel = response["results"][0]
        ideal_cities.loc[index, 'Hotel name'] = hotel["name"]
    except IndexError:
        print(f"Hmm, no hotel found in {row['City']}")
        print(response)
    except KeyError:
        print(f"Hmm, no hotel found in {row['City']}")
        print(response)

ideal_cities

<ipython-input-13-2155e7b7a53a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities['Hotel name'] = ""
C:\Users\mail\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hmm, no hotel found in cortes
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Hmm, no hotel found in ankazobe
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
26,lata,40.1629,-8.3327,74.35,40.0,0.0,5.97,PT,1.627466e+09,Quinta do Outeiro
168,dingli,35.8614,14.3822,79.66,57.0,0.0,6.49,MT,1.627466e+09,The Xara Palace Relais & Chateaux
192,cortes,41.9395,-1.4396,76.03,49.0,0.0,8.93,ES,1.627466e+09,
230,morondava,-20.2833,44.2833,78.44,37.0,0.0,8.23,MG,1.627466e+09,Palissandre Cote Ouest resort & SPA
297,bueu,42.3246,-8.7850,74.35,56.0,0.0,5.75,ES,1.627466e+09,Hotel Alda Bueumar
361,komatipoort,-25.4332,31.9548,75.06,26.0,0.0,8.25,ZA,1.627466e+09,Shishangeni by BON Hotels
550,ankazobe,-18.3167,47.1167,70.21,21.0,0.0,7.70,MG,1.627466e+09,


In [14]:
# sample code used from starter notebook
# country removed as not stored in dataframe

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
hover_text = [row['Hotel name'] for index,row in ideal_cities.iterrows()]
locations = ideal_cities[["Lat", "Lon"]]

hotel_marks = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hover_text)
fig.add_layer(hotel_marks)
fig

Figure(layout=FigureLayout(height='420px'))